### Getting started with the Ollama library ###

In [ ]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from textwrap import dedent
import logging
import datetime
from pathlib import Path

# Ollama
import ollama
from pydantic import BaseModel

# Financial news data for this notebook
from gnews import GNews

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
# print(f'Package version: {llmt.__version__}')

In [ ]:
# Data directory and files
data_dir = os.path.join(os.environ.get('HOME'), 'data')
log_dir = os.path.join(data_dir, 'log')
Path(log_dir).mkdir(exist_ok=True, parents=True)
print(f'Data: {data_dir}')
print(f'Logs:  {log_dir}')

# Log file
# Set up logger
date_str = datetime.date.today().strftime('%y%m%d')
log_file_name = f'ollama_log_{date_str}.log'
log_file = os.path.join(log_dir, log_file_name)
dtfmt = '%y%m%d-%H:%M'
logfmt = '%(asctime)s-%(name)s-%(levelname)s-%(message)s'

logging.basicConfig(filename=log_file,
                    filemode='w',
                    level=logging.INFO,
                    format=logfmt,
                    datefmt=dtfmt)

logger = logging.getLogger(name=__name__)
logger.setLevel(logging.CRITICAL)

In [ ]:
# Create a connection to Ollama
client = ollama.Client(host='http://ollama:11434')
try:
    client = ollama.Client(host='http://ollama:11434')
    models = client.list().models
    if models:
        print("Available models:")
        for model in models:
            print(f"- {model['model']}")
    else:
        print("No models found.")
except Exception as e:
    print(f"Error: {e}")

### Ollama getting started ###

In [ ]:
# Fetch new articles
google_news = GNews()
news = google_news.get_news('NVDA')

In [ ]:
# Extract a few news titles
news_titles = [article.get('title') for article in news[:6]]
print(*news_titles, sep='\n\n')

In [ ]:
class NewsAnalysis(BaseModel):
    sentiment: str
    future_looking: bool

system_message = f"""You are a powerful AI that can analyze the sentiment of news messages and give advice on specific stocks."""
model = 'llama3.2'

for title in news_titles:
    messages = [{'role': 'user',
                 'content': 
                 f"""Analyze the following title for sentiment (positive, negative, or neutral)
                 and whether it provides future-looking financial insight, predictions, or guidance on 
                 whether to buy/hold/sell the stock (True or False): {title}
                 """,}]
    response = client.chat(messages=messages, model=model, format=NewsAnalysis.model_json_schema())
    # Parse the response into the NewsAnalysis model
    sentiment_analysis = NewsAnalysis.model_validate_json(response['message']['content'])